# Batch Mode Demo Script

This is a Jupyter Notebook working through the batch processing of images in the CanopyOpenness package
It shows the user how to use the script

## This script
Runs BatchRun class object with instructions for user.

  - 1) Get input of a directory of images from user and where dataframe will be saved
  - 2) Runs BatchRun class object which:
      - Lists of all jpg files within that directory
      - Creates empty dataframe with image information (plot, subplot, date, exposure)
      - Iterates through folder, running ImagePrep, FishEye, and CanOpen modules
      - Saves openness metric as last column in the dataframe
      - Saves dataframe as a csv in folder given by user
  
 -----
 -----

## Setup

Loading packages, setting logger levels

In [1]:
#Importing my package
import CanopyOpenness
from CanopyOpenness import ImageLoad
from CanopyOpenness import FishEye
from CanopyOpenness import CanOpen
from CanopyOpenness import BatchRun 
#Importing packages
import glob #helping identify files in pathfiles
import os #finding pathfiles
import pathlib #getting pathfiles
import pandas as pd #pandas for dataframe manipulation
import numpy as np #statistical calculations
import natsort #batch loading of files
import skimage #image manipulation
from skimage import io #filepaths in skimage
import warnings #warnings package
from loguru import logger #Logger for debugging messages

In [2]:
# Setting logging level
CanopyOpenness.set_loglevel("DEBUG")

## Running BatchRun class object for batch processing

In [16]:
# Defining user input
dirpath = './sample_photos/LFDP/2012/' #where photos are stored
filepath = './Data/' #where dataframe will be saved
filename = 'LFDP_2012.csv' #name of dataframe to be saved

In [17]:
# Running BatchRun class
#    user inputs where images are stored, 
#    whether they want to save the dataframe output (defaults to false)
#    and if so, where the dataframe is saved and what it's called

doit = BatchRun.BatchRun(dirpath,filepath,filename,save=True)

In [18]:
# Checking that the batch function worked
df = doit.Batch()

# Show dataframe
df

IndexError: index 2592 is out of bounds for axis 1 with size 2592

In [6]:
# Checking saving function works
df2 = doit.SaveDF() #running function 

# List files in Data folder to check saving worked
data_files = [f for f in os.listdir(filepath) if f.endswith('csv')]

# Show folder files
data_files #works (BatchTest2.csv)

['BatchTest2.csv', 'LFDP_2017.csv', 'Batch_Test_Dataframe.csv']

---
---

# Depreceated code showing function

---
---

## Getting image files from directory

Getting user input (filepath for directory containing image files.  
Then saving each individual image as an item in a list.  
Then creating a dataframe with metadata from image names, and an empty column to hold canopy openness values

In [14]:
#Defining user input
# filepath = './sample_photos/LFDP/2012/' #where photos are stored

In [15]:
# #iterate through all images in the folder and save as a list
# all_images = [f for f in os.listdir(filepath) if f.endswith('JPG')]

# #sort the list
# all_images.sort()

# #check
# all_images

['40.100.11May2012.EV-2.MF.JPG',
 '40.100.11May2012.EV-4.MF.JPG',
 '40.100.11May2012.EV0.MF.JPG',
 '40.139.01May2012.EV-2.AF.JPG',
 '40.139.01May2012.EV-4.AF.JPG',
 '40.139.01May2012.EV0.AF.JPG',
 '40.142.04May2012.EV-2.AF.JPG',
 '40.142.04May2012.EV-4.AF.JPG',
 '40.142.04May2012.EV0.AF.JPG',
 '40.145.07May2012.EV-2.MF.JPG',
 '40.145.07May2012.EV-4.MF.JPG',
 '40.145.07May2012.EV0.MF.JPG',
 '40.148.08May2012.EV-2.AF.JPG',
 '40.148.08May2012.EV-4.AF.JPG',
 '40.148.08May2012.EV0.AF.JPG',
 '40.151.11May2012.EV-2.AF.JPG',
 '40.151.11May2012.EV-4.AF.JPG',
 '40.151.11May2012.EV0.AF.JPG',
 '40.190.01May2012.EV-2.MF.JPG',
 '40.190.01May2012.EV-4.MF.JPG',
 '40.190.01May2012.EV0.MF.JPG',
 '40.193.04May2012.EV-2.AF.JPG',
 '40.193.04May2012.EV-4.AF.JPG',
 '40.193.04May2012.EV0.AF.JPG',
 '40.196.07May2012.EV-2.MF.JPG',
 '40.196.07May2012.EV-4.MF.JPG',
 '40.196.07May2012.EV0.MF.JPG',
 '40.199.08May2012.EV-2.AF.JPG',
 '40.199.08May2012.EV-4.AF.JPG',
 '40.199.08May2012.EV0.AF.JPG',
 '40.202.11May2012.E

In [12]:
# # create an Empty DataFrame object
# df = pd.DataFrame(columns=['Plot','Subplot','Date','Exposure','Openness'])

# #split image by period to get different columns
# df['Plot'] = [i.split('.')[0] for i in all_images]  # get first item in list of images
# df['Subplot'] = [i.split('.')[1] for i in all_images]  # get subplot information (second item)
# df['Date'] = [i.split('.')[2] for i in all_images]  # get date information (third item)
# df['Exposure'] = [i.split('.')[3] for i in all_images]  # get exposure information (fourth item)

# #check
# df

,Plot,Subplot,Date,Exposure,Openness
0,40,100,11May2012,EV-2,NaN
1,40,100,11May2012,EV-4,NaN
2,40,100,11May2012,EV0,NaN
3,40,139,01May2012,EV-2,NaN
4,40,139,01May2012,EV-4,NaN
...,...,...,...,...,...
117,40,94,07May2012,EV-4,NaN
118,40,94,07May2012,EV0,NaN
119,40,97,08May2012,EV-2,NaN
120,40,97,08May2012,EV-4,NaN


## Batch processing function

Code to create an empty list to hold openness values.  
Then iterating over all the image files and running ImagePrep, FishEye, and CanOpen modules to calculate openness for each image.  
Then storing those values in the empty column in the dataframe containing the image metadata.  
Finally storing the resultant dataframe as a csv for the user

In [13]:
# #create empty list to hold results
# results = []

# #iterate through list of images and run CanopyOpenness functions
# for image in all_images:
    
#     #load image and threshold using isodata algorithm, don't plot, set to batch
#     img = ImageLoad.ImagePrep(filepath,image,threshold_method="isodata",plot=False,batch=True)
#     #load image
#     og = img.imageLoad()
#     #turn blue 
#     blue = img.BluePic()
#     #threshold algorithm and turn to black and white
#     bw = img.bwPic()
    
#     #set fisheye coordinates for center lens, don't plot, set to batch
#     fish = FishEye.FishEye(bw,plot=False,batch=True)
#     #save image array with coordinates
#     fishy = fish.CircleCoords()
    
#     #run canopy openness module, set to batch
#     gfp = CanOpen.CanOpen(fishy,batch=True) #running module
#     gaps=gfp.calc_gap_fractions() #calculating array of proportion sky for 89 sub-circles within fisheye lens
#     openness=gfp.openness() #openness calculation
    
#     #print message to user
#     print("Image",image, "processed")

#     #appending to result list
#     results.append(openness)

IndexError: index 2592 is out of bounds for axis 1 with size 2592

In [ ]:
# #add values from result to the dataframe
# df['Openness'] = results

# #check
# df

In [ ]:
# #saving dataframe as a csv
# df.to_csv(r'./Data/Batch_Test_Dataframe.csv', index = False)